In [2]:
import get_price.get_futures_price as gfp
data_out = gfp.get_futures_price_preloaded(ticker_head='LN',settle_date=20161216)
data_out = data_out[data_out['tr_dte']>30]
data_out.sort(['cont_indx'],inplace=True)

ticker_list = [data_out['ticker'].iloc[0],data_out['ticker'].iloc[1]]
ticker_list

['LNG2017', 'LNJ2017']

In [22]:
import opportunity_constructs.utilities as opu
aligned_output = opu.get_aligned_futures_data(contract_list=ticker_list,aggregation_method=12,contracts_back=10,
                                              date_to=20161216,
                                              tr_days_half_band_width_selected=100,
                                              get_uniqe_data = False)

aligned_data = aligned_output['aligned_data']

aligned_data['spread_price'] = aligned_data['c1']['close_price']-aligned_data['c2']['close_price']
aligned_data['spread_change5'] = (aligned_data['c1']['change5']-aligned_data['c2']['change5'])/aligned_data['c1']['close_price']



import matplotlib.pyplot as plt
plt.plot(aligned_data['c1']['cont_indx'])
plt.grid()         
         
plt.show()

In [26]:
aligned_data['cont_indx'] = aligned_data['c1']['cont_indx']
grouped = aligned_data.groupby('cont_indx')
aligned_data['ma45'] = grouped['spread_price'].apply(pd.rolling_mean,45)
aligned_data['ma15'] = grouped['spread_price'].apply(pd.rolling_mean,15)
aligned_data['ppo'] = (aligned_data['ma15']-aligned_data['ma45'])/aligned_data['c1']['close_price']

import matplotlib.pyplot as plt
plt.plot(range(len(aligned_data.index)),aligned_data['spread_price'],
         range(len(aligned_data.index)),aligned_data['ma45'],
         range(len(aligned_data.index)),aligned_data['ma15'])
plt.grid()         
         
plt.show()

In [107]:
selected_data = aligned_data[aligned_data['ma15']<aligned_data['ma45']]


import matplotlib.pyplot as plt
plt.plot(selected_data['spread_change5'].cumsum())
plt.grid()         
         
plt.show()

In [30]:
aligned_data = aligned_data[aligned_data['ppo'].notnull()]
aligned_data = aligned_data[aligned_data['spread_change5'].notnull()]

In [99]:
from sklearn import svm
import numpy as np
model = svm.SVC(kernel='linear', C=100., gamma=0.001) 

model.fit(np.array(aligned_data['ppo'].values).reshape((852,1)), np.array(400*[40]+452*[30]))



SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='linear', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [104]:
model.predict(aligned_data['ppo'].values[370])

array([30])

In [85]:
digits = datasets.load_digits()
clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(digits.data[:-1], digits.target[:-1])  

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [97]:
digits.target[:-1]

array([0, 1, 2, ..., 0, 8, 9])

In [87]:
clf.predict(digits.data[1:])

array([1, 2, 3, ..., 8, 9, 8])